# Deploy model

In [1]:
#!pip install -U hopsworks --quiet

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550038
Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;"> Deploy model</span>

In [4]:
%%writefile predict_example.py
import os
import numpy as np
import hsfs
import joblib

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""
        # load the trained model
        self.model = joblib.load(os.environ["ARTIFACT_FILES_PATH"] + "/xgboost.pkl")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request using a trained model"""
        # Convert inputs to numpy array to handle batch predictions
        inputs_array = np.array(inputs)
        # Check if the input is a single instance or batch
        if inputs_array.ndim == 1:
            # Reshape if a single instance to match input shape expected by the model
            inputs_array = inputs_array.reshape(1, -1)
        # Make predictions
        return self.model.predict(inputs_array).tolist()  # Return predictions as list

Overwriting predict_example.py


In [5]:
import os
dataset_api = project.get_dataset_api()

uploaded_file_path = dataset_api.upload("predict_example.py", "Models", overwrite=True)
predictor_script_path = os.path.join("/Projects", project.name, uploaded_file_path)

Uploading: 100.000%|██████████| 885/885 elapsed<00:01 remaining<00:00


In [6]:
print("Uploaded file path:", uploaded_file_path)
print("Predictor script path:", predictor_script_path)


Uploaded file path: Models/predict_example.py
Predictor script path: /Projects/skris20/Models/predict_example.py


## Create the deployment

In [8]:
# Use the model name from the previous notebook.
mr = project.get_model_registry()
model = mr.get_model("BeerRatingPredictionModel1", version=1)

# Create and start the deployment
deployment = model.deploy(
    name="beerratingmodeldeployment1",  # Give it a descriptive name
    serving_tool="KSERVE",  # Specify the serving tool (e.g., KSERVE)
    script_file=predictor_script_path  # The path to your prediction script
)

print("Deployment: " + deployment.name)
deployment.describe()
deployment.start()

Connected. Call `.close()` to terminate connection gracefully.
Deployment with the same name already exists. Getting existing deployment...
To create a new deployment choose a different name.
Deployment: beerratingmodeldeployment1
{
    "artifact_version": 1,
    "batching_configuration": {
        "batching_enabled": false
    },
    "created": "2024-05-07T03:52:59.000Z",
    "creator": "Sebastian Kristensen",
    "description": null,
    "id": 257029,
    "inference_logging": "NONE",
    "model_framework": "PYTHON",
    "model_name": "BeerRatingPredictionModel1",
    "model_path": "/Projects/skris20/Models/BeerRatingPredictionModel1",
    "model_server": "PYTHON",
    "model_version": 1,
    "name": "beerratingmodeldeployment1",
    "predictor": "predict_example.py",
    "predictor_resources": {
        "limits": {
            "cores": 0.5,
            "gpus": 0,
            "memory": 1024
        },
        "requests": {
            "cores": 0.2,
            "gpus": 0,
            "